In [ ]:
%matplotlib inline
import seaborn
from matplotlib import pyplot as plt
import matplotlib
seaborn.set(style="whitegrid")
seaborn.set_context('poster')

In [ ]:
import simulate
import withdrawal
import market
import harvesting
import metrics

from decimal import Decimal as D
import decimal
import itertools
import pandas
import numpy

In [ ]:
def run(market, stock_pct=D('.5'), length=30):
    start_year = market.start_year
    last_year = market.last_year - length
    swrs = pandas.Series()
    for _ in range(last_year - start_year + 2):
        returns = []
        for one_year in zip(range(length), market.iter_from(start_year + _)):
            annual_returns = one_year[1]
            stocks = ((1+annual_returns.stocks) / (1+annual_returns.inflation)) - 1
            bonds = ((1+annual_returns.bonds) / (1+annual_returns.inflation)) - 1
            returns.append((stocks * stock_pct) + (bonds * (1-stock_pct)))
        swr = float(metrics.pwa(100, 0, returns))
        swrs.loc[start_year+_] = swr
    return swrs

In [ ]:
def plot(swrs, title):
    plt.figure(figsize=(20,10)) 
    ax = seaborn.lineplot(
                    data=swrs,
                   )
    min = swrs['SWR'].min()
    seaborn.despine(ax=ax, left=True, bottom=True, offset=20)
    ax.set_title(f'SWR in {title}. Min = {min:.1f}')

In [ ]:
for country in market.JST.Countries:
    if country == 'CAN': continue
    m = market.JST(country)
    swrs = run(m)
    df = pandas.DataFrame(swrs, columns=['SWR'])
    plot(df, country)

In [ ]:
MSWRS = pandas.Series(index=market.JST.Countries)

for country in market.JST.Countries:
    if country == 'CAN': continue
    m = market.JST(country)
    m.start_year = 1950
    swrs = run(m)
    mswr = swrs.min()
    MSWRS.loc[country] = mswr

In [ ]:
MSWRS.sort_values(ascending=False)